In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

def scroll_to_bottom(driver, scroll_pause_time=2):
    """페이지의 끝까지 스크롤하여 모든 댓글을 로드합니다."""
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        # 새로운 콘텐츠가 로드될 때까지 기다림
        time.sleep(scroll_pause_time)
        
        # 새로운 높이 계산
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # 더 이상 스크롤되지 않으면 종료
        if new_height == last_height:
            break
        last_height = new_height

def get_youtube_shorts_comments(url):
    """
    유튜브 숏츠 URL에서 댓글을 추출합니다.
    """
    chrome_options = Options()
    # 브라우저를 눈으로 보고 싶지 않다면 다음 주석을 해제하세요.
    # chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--log-level=3") # INFO, WARNING 등을 숨김

    # ChromeDriver 경로 설정 (다운로드한 ChromeDriver의 경로로 변경)
    # service = Service(executable_path="C:/path/to/chromedriver.exe") 
    # 위 라인을 사용하거나, ChromeDriver가 PATH에 추가되어 있다면 service=Service()만 사용
    service = Service() # ChromeDriver가 PATH에 있다면 이렇게 사용

    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(30) # 페이지 로드 타임아웃 설정

    comments_list = []

    try:
        print(f"Opening URL: {url}")
        driver.get(url)
        time.sleep(5) # 페이지 로드 및 초기 요소들이 나타날 시간을 충분히 줌

        # 숏츠는 댓글이 바로 보이는 경우가 있지만, iframe 내부에 있을 수도 있습니다.
        # 일반 유튜브 동영상과 동일하게 댓글 섹션을 찾습니다.
        print("Scrolling to comments section...")
        # 댓글 섹션으로 스크롤하기 위한 특정 요소 (예: 댓글 카운트 또는 첫 댓글)
        try:
            # 유튜브 댓글 섹션의 ID를 찾습니다.
            driver.execute_script("window.scrollTo(0, 800);") # 초기 스크롤
            time.sleep(2)
            
            # 'comments' ID를 가진 섹션을 찾거나, 더 일반적인 댓글 섹션의 컨테이너를 찾습니다.
            # 숏츠의 경우 댓글이 좀 더 명확하게 댓글 아이콘 클릭 후 로드되는 경우가 많습니다.
            # 먼저 댓글 아이콘을 클릭해야 할 수 있습니다.
            
            # 숏츠의 경우 댓글 아이콘을 클릭하여 댓글 섹션을 열어야 합니다.
            # 이 부분은 유튜브 UI 변경에 따라 XPath가 변경될 수 있습니다.
            # 현재 숏츠의 댓글 버튼 XPath (경험상 이 위치에 자주 있습니다.)
            try:
                # 댓글 아이콘 버튼 (클릭하여 댓글 패널을 열어야 함)
                # XPath는 변경될 수 있으므로, 웹에서 직접 확인하는 것이 가장 정확합니다.
                comment_button_xpath = '//*[@id="comments-button"]' # 숏츠 전용 댓글 버튼 ID
                comment_button = driver.find_element(By.XPATH, comment_button_xpath)
                comment_button.click()
                print("Clicked comment button.")
                time.sleep(3) # 댓글 패널이 열릴 시간을 줍니다.
                
                # 댓글 패널이 iframe 안에 있다면 전환해야 합니다.
                # 그러나 대부분의 숏츠 댓글은 페이지 내부에 바로 로드됩니다.
                
            except NoSuchElementException:
                print("Comment button not found or already open. Proceeding to scroll.")
            except Exception as e:
                print(f"Error clicking comment button: {e}")
            
            # 댓글 패널이 열린 후 댓글 영역을 찾아 스크롤합니다.
            # 유튜브 댓글 섹션은 'comments' ID를 가진 경우가 많습니다.
            # 또는 'ytd-comments-renderer'와 같은 클래스/태그를 가진 요소를 찾을 수 있습니다.
            try:
                comments_section = driver.find_element(By.ID, "comments")
                driver.execute_script("arguments[0].scrollIntoView();", comments_section)
                print("Scrolled to comments section (ID: comments).")
                time.sleep(2)
            except NoSuchElementException:
                print("Comments section with ID 'comments' not found. Trying alternative scroll.")
                # 댓글 섹션이 특정 ID 없이 로드될 경우, 전체 페이지 스크롤로 대체
                pass
            
            # 모든 댓글이 로드될 때까지 스크롤
            print("Scrolling to load all comments...")
            scroll_to_bottom(driver)
            print("Finished scrolling.")

            # 댓글 추출
            # 댓글 내용과 작성자 이름을 포함하는 요소들을 찾습니다.
            # 이 XPath도 유튜브 UI 변경에 따라 변경될 수 있습니다.
            comment_elements = driver.find_elements(By.XPATH, '//*[@id="comment"]')
            
            if not comment_elements:
                print("No comment elements found with the specified XPath.")
                # 다른 XPath 시도 (예: 일반 유튜브 댓글의 XPath)
                comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]') # 댓글 내용
                if comment_elements:
                    print(f"Found {len(comment_elements)} comments using 'content-text' XPath.")
                else:
                    print("No comments found after trying multiple XPaths.")


            for comment_element in comment_elements:
                try:
                    author_element = comment_element.find_element(By.XPATH, './/*[@id="author-text"]/span')
                    text_element = comment_element.find_element(By.XPATH, './/*[@id="content-text"]')
                    
                    author = author_element.text.strip() if author_element else "N/A"
                    comment_text = text_element.text.strip() if text_element else "N/A"
                    
                    comments_list.append({"author": author, "comment": comment_text})
                except NoSuchElementException:
                    # 일부 댓글 요소에서 특정 부분이 없을 수 있음
                    continue
                except Exception as e:
                    print(f"Error extracting parts of a comment: {e}")
                    continue

        except NoSuchElementException:
            print("Could not find the initial comments container. Check the URL or XPath.")
            # 페이지 로드 후에도 댓글이 나타나지 않을 경우.
        except TimeoutException:
            print("Page load timed out.")
            
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
        print("Browser closed.")
    
    return comments_list

# 사용 예시
if __name__ == "__main__":
    shorts_url = "https://www.youtube.com/shorts/PFiyCYfNn80" # 여기에 실제 숏츠 URL을 넣으세요.
    # 예시: https://www.youtube.com/shorts/dQw4w9WgXcQ (유명한 숏츠는 아니지만 예시)

    comments = get_youtube_shorts_comments(shorts_url)

    if comments:
        print(f"\n--- Found {len(comments)} Comments ---")
        for i, comment_data in enumerate(comments):
            print(f"{i+1}. Author: {comment_data['author']}")
            print(f"   Comment: {comment_data['comment']}\n")
    else:
        print("No comments found or an error occurred.")

Opening URL: https://www.youtube.com/shorts/PFiyCYfNn80
Scrolling to comments section...
Clicked comment button.
Scrolled to comments section (ID: comments).
Scrolling to load all comments...
Finished scrolling.
Browser closed.

--- Found 20 Comments ---
1. Author: @gae_zib
   Comment: ”모든 단군할아버지의 백성에게 전한다“

2. Author: @Fish-j7i
   Comment: “내가 돌하르방이고 이녀석이 모아이석상이야“

3. Author: @댓라이프
   Comment: 원본 댓글 중 개웃겼던게ㅋㅋㅋㅋㅋ
"모든 시조 웅녀의 자손들에게 전한다"
"모두 구축해주겠땅게"
"모두 몰아내주겠다......-태조 이성계"

4. Author: @김인숙-m7o
   Comment: "내 김치를 담가줘서 고마워"

5. Author: @이잉아-r2d
   Comment: 모르는 사람들이 많은데 저건 옛날이야기 알려주는 애니 배추도사 무도사의
 '울산바위 이야기'편이고 저 장면은 옥황상제가 금강산을 만들건데 들어가서 금강산의 일부가 되고싶어하는 전국각지의 돌들을 선착순으로 모집해서 다들 저렇게 뛰어가는거임ㅋㅋ 오랜만이네

6. Author: @Ppap-q4x
   Comment: 이건 단군 할아버지가 시작한 이야기잖아..

7. Author: @JACKy1459-x9t
   Comment: 그날 길동은 떠올렸다

8. Author: @혜성님
   Comment: 독립해 주겠어! 한곳도 남김없이!

9. Author: @ONfan_썸머STAR_l9금체널
   Comment: 조상의 유산을 잊지 말고 함께 일어섭시다

10. Author: @U4HP1
   Comment: "장군 한 명을 조심하라고?"
"네, 이순신 장군은 위험합니다."

11. A